# Flying a Drone Through a Pendulum Hoop

When dispatching autonomous drones into dynamic situations, users need to be able to trust that their drones can dynamically track objects in their surroundings and update their desired positions accordingly. In this project, we aim to model a simplified version, but simulating a swinging pendulum and a drone which must use the dynamics of the swinging pendulum-hoop and real-time object tracking to update its estimated position of the system and drive so it passes through the hoop at the perfect time. This project requires a combination of motion planning, perception, and control, adding a new level of challenge to existing work. 

Currently, there are no pre-existing methods driving a drone through a swinging pendulum. There are two major components to this project: determining the drone-pendulum intersection point, and controlling the drone to approach that point at the correct time.

In [ ]:
import math
import cv2
import pydot
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import os
from IPython.display import HTML, display, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    StartMeshcat,
    Propeller,
    PropellerInfo,
    RigidTransform,
    RobotDiagramBuilder,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
    namedview,
    VectorLogSink,
    VectorSystem,
    wrap_to,
    MultibodyPositionToGeometryPose,
    ModelInstanceIndex,
    RenderCameraCore,
    ClippingRange,
    DepthRange,
)
from pydrake.all import RigidTransform, AddMultibodyPlantSceneGraph, DiagramBuilder, Simulator, MeshcatVisualizer, MeshcatVisualizerParams, SceneGraph, LeafSystem, Diagram, AbstractValue
from pydrake.all import MultibodyPlant, Parser, FindResourceOrThrow, Meshcat, RigidTransform, MakeRenderEngineVtk
from pydrake.examples import (
    AcrobotGeometry,
    AcrobotInput,
    AcrobotPlant,
    AcrobotState,
    QuadrotorGeometry,
    QuadrotorPlant,
    StabilizingLQRController,
)
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.geometry import (
    MeshcatVisualizerParams,
    Role,
)
from underactuated import ConfigureParser, running_as_notebook
from underactuated.meshcat_utils import MeshcatSliders
from underactuated.quadrotor2d import Quadrotor2D, Quadrotor2DVisualizer
from underactuated.scenarios import AddFloatingRpyJoint

from copy import deepcopy
from pydrake.examples import PendulumGeometry, PendulumParams, PendulumPlant

from underactuated import running_as_notebook

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer

import dataclasses as dc
from functools import partial
import os
import sys
import typing

import torch
import torch.utils.data
import torchvision
import torchvision.transforms.functional as Tf

from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

import numpy as np
from math import atan2, cos, sin, sqrt, pi

import time

from pydrake.all import (
    AbstractValue,
    Adder,
    AddMultibodyPlant,
    ApplyLcmBusConfig,
    ApplyMultibodyPlantConfig,
    ApplyVisualizationConfig,
    BaseField,
    CameraConfig,
    CameraInfo,
    Demultiplexer,
    DepthImageToPointCloud,
    Diagram,
    DiagramBuilder,
    DrakeLcmParams,
    GetScopedFrameByName,
    IiwaCommandSender,
    IiwaDriver,
    IiwaStatusReceiver,
    InverseDynamicsController,
    LeafSystem,
    LcmPublisherSystem,
    LcmSubscriberSystem,
    MakeMultibodyStateToWsgStateSystem,
    MakeRenderEngineVtk,
    RenderEngineVtkParams,
    Meshcat,
    MeshcatPointCloudVisualizer,
    ModelDirective,
    ModelDirectives,
    MultibodyPlant,
    MultibodyPlantConfig,
    OutputPort,
    Parser,
    PassThrough,
    ProcessModelDirectives,
    RgbdSensor,
    RigidTransform,
    SceneGraph,
    SchunkWsgDriver,
    SchunkWsgPositionController,
    SchunkWsgCommandSender,
    SchunkWsgStatusReceiver,
    ScopedName,
    SimulatorConfig,
    StateInterpolatorWithDiscreteDerivative,
    VisualizationConfig,
    ZeroForceDriver,
    DepthRenderCamera,
)
from pydrake.common.yaml import yaml_load_typed
from drake import (
    lcmt_iiwa_command,
    lcmt_iiwa_status,
    lcmt_schunk_wsg_command,
    lcmt_schunk_wsg_status,
)

if running_as_notebook:
    mpld3.enable_notebook()

In [ ]:
from pydrake.all import StartMeshcat
from pydrake.examples import StabilizingLQRController
current_time = None
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://d095d7c3-1038-4cc5-bfcb-77f17ef5665a.deepnoteproject.com/7000/


# Finding the Intersection Point Between Drone and Pendulum

In this section, we focus on motion planning and retrieving the final point where the drone flies through a pendulum hoop at a designated time. For this, we have three sections: Estimating the desired point for the quadrotor to go to through modeling the dynamics of the pendulum, tracking the location of the pendulum to verify the system dynamics, and using a Kalman filter to mesh together both of those pieces of information.

## Estimating Pendulum Position through Dynamics Modeling

In [ ]:
g = 9.8 # gravity constant
l = 1.0 # length of the pendulum
b = 0.0 # damping factor for the pendulum
max_angle = np.radians(90) # maximum angle the pendulum can rise to

# This function models the pendulum equation through the Lagrangian Derivation
def damped_pendulum_equation(theta, t, g, l, b):
    dtheta_dt = theta[1]
    d2theta_dt2 = -b * theta[1] - (g / l) * np.sin(theta[0])
    return [dtheta_dt, d2theta_dt2]

initial_conditions = [max_angle, 0.0] # theta, angular velocity

# This function solves the ODE modeling the pendulum to retrieve the theta at time t 
def get_theta_at_time(t):
    t = np.linspace(0, t, 2)
    return odeint(damped_pendulum_equation, initial_conditions, t, args=(g, l, b))[:, 0][-1]


In [ ]:
# Given a designated time, this function retrieves the position of the pendulum at that time

def get_coords_at_time(length, t):
    theta_at_time = get_theta_at_time(t)
    x = length * np.cos(theta_at_time)
    if theta_at_time < 0:
        x = -x
    y = -length * np.abs(np.sin(theta_at_time))
    z = dist_from_drone
    return [x, z, y]

[0.26289580876382973, 0.1, -0.7555698470256816]


# Trajectory Optimization/Control

This section is all about controlling the drone to drive to the designated point, and also includes the code to render the pendulum in Meshcat.

## Using LQR Controller to Drive Drone to a Designated Point

This controller uses LQR and has no inclusion of timing. It is used as a preliminary method used, and is used in our baseline, which does not consider the time of intersection between the drone and the pendulum hoop, and simply drives to the designated point immediately after spawning.

In [ ]:
from pydrake.math import RotationMatrix

def simulation(x):

    # This simulation contains a swinging pendulum and drives the drone to
    # designated point specified through the methods above

    builder = DiagramBuilder()

    pendulum = builder.AddSystem(PendulumPlant())
    plant = builder.AddSystem(QuadrotorPlant())

    # Quadrotor Controller
    controller = builder.AddSystem(StabilizingLQRController(plant,x))
    builder.Connect(controller.get_output_port(0), plant.get_input_port(0))
    builder.Connect(plant.get_output_port(0), controller.get_input_port(0))

    # Setup visualization
    scene_graph = builder.AddSystem(SceneGraph())
    PendulumGeometry.AddToBuilder(
        builder, pendulum.get_state_output_port(), scene_graph
    )
    QuadrotorGeometry.AddToBuilder(
        builder, plant.get_output_port(0), scene_graph
    )
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Delete()
    meshcat.ResetRenderMode()
    meshcat.SetProperty("/Background", "visible", False)
 
    # Setup slider input for pendulum input torque
    meshcat.AddSlider("u", min=-5, max=5, step=0.1, value=0.0)
    torque_system = builder.AddSystem(MeshcatSliders(meshcat, ["u"]))
    builder.Connect(torque_system.get_output_port(), pendulum.get_input_port())

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    pend_context = pendulum.GetMyMutableContextFromRoot(context)
    params = pendulum.get_mutable_parameters(context=pend_context)
    params.set_damping(damping=0)
    params.set_length(length=3)

    meshcat.AddButton("Stop Simulation")

    # Set the initial conditions for an array with the pendulum initial state and the quadrotor initial state
    init_state = np.concatenate((np.array([np.pi/2, 0]), 0.5 * np.random.randn(12,)), axis = 0)
    context.SetContinuousState(
            init_state
        )

    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)

        print("Use the slider in the MeshCat controls to apply elbow torque.")
        print("Press 'Stop Simulation' in MeshCat to continue.")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)
    else:
        simulator.AdvanceTo(0.1)

    meshcat.DeleteAddedControls()

#This function drives the drone to intersect with the hoop using LQR

def lqr_drive(t):
    pend_pos = get_coords_at_time(t)
    drone_desired_pose = pend_pos
    simulation(drone_desired_pose)

## Visual Pose Estimation

Using the cameras in the simulation, we extract RGB images and run some basic feature extraction techniques to extract the large red rectangular contour representing the pendulum rod. The orientation of this contour is used to inform the pendulum's current angle. 

In [ ]:
class pose_estimation():
    def __init__(self):
        meshcat.Delete()
        meshcat.DeleteAddedControls()
        meshcat.ResetRenderMode()
        meshcat.SetProperty("/Background", "visible", False)

        builder = DiagramBuilder()
        
        # set up the system of manipulation station
        self.station = MakeManipulationStation(filename="package://models/sim.dmd.yaml", package_xmls=["models/package.xml"]) 

        builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")
        
        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(builder, self.station.GetOutputPort("query_object"), meshcat, params)

        scene_graph = builder.AddSystem(SceneGraph())
        MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

        self.diagram = builder.Build()
        self.world_frame = self.plant.world_frame()
        context = self.CreateDefaultContext()
        
        #################### CAMERA ########################
        diagram_context = self.station.CreateDefaultContext() #get the context for the entire manipulation station. 

        depth_im_read = self.station.GetOutputPort("camera0_depth_image").Eval(diagram_context).data.squeeze() #change to camera_context 
        self.depth_im = deepcopy(depth_im_read)
        self.depth_im[self.depth_im == np.inf] = 10.0
        self.rgb_im = self.station.GetOutputPort("camera0_rgb_image").Eval(diagram_context).data

        self.cam = self.station.GetSubsystemByName("camera0")
        cam_context = self.cam.GetMyMutableContextFromRoot(context)
        self.X_WC = self.cam.body_pose_in_world_output_port().Eval(cam_context)
        self.X_WC = RigidTransform(self.X_WC)
        self.cam_info = self.cam.depth_camera_info()

    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(self.plant, context)
        station_context = self.diagram.GetMutableSubsystemContext(self.station, context)
        return context

    def viz(self, sim_duration=1.0):
        # Set up a simulator to run this diagram
        simulator = Simulator(self.diagram)
        simulator.Initialize()
        if running_as_notebook:
            simulator.set_target_realtime_rate(1.0)
            simulator.AdvanceTo(sim_duration)
        else:
            simulator.AdvanceTo(0.1)
 
# Find the orientation of the extracted contour
def getOrientation(pts, img):
  sz = len(pts)
  data_pts = np.empty((sz, 2), dtype=np.float64)
  for i in range(data_pts.shape[0]):
    data_pts[i,0] = pts[i,0,0]
    data_pts[i,1] = pts[i,0,1]
 
  mean = np.empty((0))
  mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
 
  cntr = (int(mean[0,0]), int(mean[0,1]))

  p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
  p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
  angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians

  return angle

In [ ]:
# Query every second for 20 seconds to capture a few passes of the pendulum
cam_angles_obs = []
for i in range(20):
    pend = pose_estimation()
    im = pend.rgb_im
    image = cv2.cvtColor(im, cv2.COLOR_RGB2HSV)
    mask1 = cv2.inRange(image, np.array([0, 70, 50]), np.array([10, 255, 255]))
    mask2 = cv2.inRange(image, np.array([160, 70, 50]), np.array([180, 255, 255]))
    mask = mask1 + mask2

    output_img = im.copy()

    cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(im, (x, y), (x + w, y + h), [0,0,0], 2)

    for i, c in enumerate(cnts):
        area = cv2.contourArea(c)
        if area < 100 or 5000 < area:
            continue
        ang = getOrientation(c, output_img)
    
    plt.imshow(output_img)
    plt.show()  

    pend.viz()
    cam_angles_obs.append(int(np.rad2deg(ang)) - 90))

## Kalman Filtering

To combine the outputs of the dynamics model and the visual observations from the camera we add a Kalman filter into the pipeline. We can then query for the final desired positions from these filtered values (representing an updated understanding of the pendulum dynamics).

In [ ]:
# Time points to evaluate
t = np.linspace(0, 20, 1000)  # Time from 0 to 20 seconds with 1000 points

# Solve the damped pendulum ODE
solution = odeint(damped_pendulum_equation, initial_conditions, t, args=(g, l, b))

# Extract theta values from the solution
theta_values = solution[:, 0]

cam_t = np.linspace(0, 19, 20)

# Plot the results of the dynamics model (blue line), camera observations (orange points) and filtered output (orange line)
plt.plot(t, np.degrees(theta_values))
plt.scatter(cam_t, cam_angles_obs, c="red")
plt.title('Pendulum Motion')
plt.xlabel('Time (s)')
plt.ylabel('Angular Displacement (degrees)')
plt.grid(True)
plt.show()

## Direct Collocation

The next method is direct collocation, a numerical optimization technique used in trajectory optimization for dynamic systems, to determine its trajectory. It formulates the problem by discretizing the system dynamics and the control inputs over a finite set of collocation points along the trajectory. It simultaneously optimizes both the state variables and control inputs at these collocation points. This method is used to plan the drone's motion, and the produced trajectory is then fed into the simulator so it can be rendered using the given quadrotor.

In [ ]:
quadrotor = """
<?xml version="1.0"?>
<!-- Mesh file and approximate numbers are from Abe Bachrach at Skdio.  -->
<robot name="quadrotor">
  <!--
  Axes are standard vehicle coordinates:
    +X - Pointing toward the nose (primary camera).
    +Y - Towards the left rotors.
    +Z - Towards the top of the vehicle..
  Origin:
    (0, 0, 0) in the center of the main quadrotor body.
  -->
  <link name="base_link">
    <inertial>
      <mass value="0.775"/>
      <origin xyz="0 1 0"/>
      <inertia ixx="0.0015" ixy="0.0" ixz="0.0" iyy="0.0025" iyz="0.0" izz="0.0035"/>
    </inertial>
    <visual>
      <origin rpy="1.570796 0 0" xyz="0 1 0"/>
      <geometry>
        <mesh filename="package://drake_models/skydio_2/skydio_2_1000_poly.obj" scale=".00254"/>
      </geometry>
    </visual>
    <collision>
      <origin rpy="0 1 0" xyz="0 0 0"/>
      <geometry>
        <box size=".36 .4 .06"/>
      </geometry>
    </collision>
  </link>
</robot>
"""

In [ ]:
def dircol_quadrotor(final_position, time):
    # Outputs the final trajectory necessary to drive the quadrotor to the final state
    plant = QuadrotorPlant()
    context = plant.CreateDefaultContext()

    dircol = DirectCollocation(
        plant,
        context,
        num_time_samples=21,
        minimum_time_step= time/20.0, # Used to control the time at which the pendulum and drone should intersect
        maximum_time_step=0.2,
    )
    prog = dircol.prog()

    dircol.AddEqualTimeIntervalsConstraints()

    # Add input torque limit
    torque_limit = 8.0  # N*m.
    u = dircol.input()
    dircol.AddConstraintToAllKnotPoints(-torque_limit <= u[0])
    dircol.AddConstraintToAllKnotPoints(u[0] <= torque_limit)
    
    # Begins at position point [0, 0, 0] with zero velocity
    initial_state = np.zeros((12,1))
    prog.AddBoundingBoxConstraint(
        initial_state, initial_state, dircol.initial_state()
    )
   
    # Final state determined by stationary end position of quadrotor
    final_state = [final_position[0], final_position[1], final_position[2], 0, 0, 0, 0, 0, 0, 0, 0, 0]
    prog.AddBoundingBoxConstraint(
        final_state, final_state, dircol.final_state()
    )
    
    R = 10  # Cost on input "effort".
    dircol.AddRunningCost(R * u[0] ** 2)

    # Add a final cost equal to the total duration.
    dircol.AddFinalCost(dircol.time())

    # Give an initial guess from a solution.
    initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(
        [0.0, 0.2, 1.4, 2.0, 4.0],
        np.column_stack(
            (
                initial_state,
                [0,0,0,0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0,0,0,0],
                final_state,
            )
        ),
    )
  
    dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)

    solver = SnoptSolver()
    solver_id = solver.solver_id()
    major_tol = 1e-3
    minor_tol = 1e-3
    prog.SetSolverOption(solver_id, "Feasibility tolerance", major_tol)
    prog.SetSolverOption(solver_id, "Major feasibility tolerance", major_tol)
    prog.SetSolverOption(solver_id, "Major optimality tolerance", major_tol)
    prog.SetSolverOption(solver_id, "Minor feasibility tolerance", minor_tol)
    prog.SetSolverOption(solver_id, "Minor optimality tolerance", minor_tol)
    result = Solve(prog)
    assert result.is_success()
    
    # Generating trajectory of input torques
    u_trajectory = dircol.ReconstructInputTrajectory(result)
    times = np.linspace(
        u_trajectory.start_time(), u_trajectory.end_time(), 100
    )
    u_values = u_trajectory.vector_values(times)
     
    # Generating trajectory of states
    x_trajectory = dircol.ReconstructStateTrajectory(result)
    x_values = x_trajectory.vector_values(times)

    return TrajectorySource(x_trajectory)


x_traj_source = dircol_quadrotor()

### Meshcat Simulator Rendering

In [ ]:
def create_scene(x_traj_source, sim_time_step=0.0001):
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    # Adding quadrotor to diagram
    builder = DiagramBuilder()
    plant = MultibodyPlant(time_step=0.0)
    scene_graph = SceneGraph()
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.AddModelsFromString(quadrotor, "urdf") # loaded from the quadrotor model above
    plant.Finalize()
   
    plant_context = plant.CreateDefaultContext()

    # Note: The quadrotor model loaded from the urdf has a 13-element state because quaternions are used 
    # for orientation instead of rpy
    plant.SetPositions(plant_context, np.zeros((7,1))) # Set all states to 0
    plant.SetVelocities(plant_context, np.zeros((6,1))) # Set all states to 0

    builder.AddSystem(x_traj_source)
    builder.AddSystem(scene_graph)
    QuadrotorGeometry.AddToBuilder(builder, x_traj_source.get_output_port(0), scene_graph)
    pos_to_pose = builder.AddSystem(
        MultibodyPositionToGeometryPose(plant, input_multibody_state=True)
    )

    # Adding pendulum to diagram
    pendulum = PendulumPlant()
    PendulumGeometry.AddToBuilder(
        builder, pendulum.get_state_output_port(), scene_graph
    )

    # Adding top-view (cam0) and frontal (cam1) cameras to diagram
    camera_files = ["models/cam0.sdf", "models/cam1.sdf"]
    for i in range(len(camera_files)):
        cam_plant = MultibodyPlant(time_step=0.0)
        cam_scene_graph = SceneGraph()
        cam_plant.RegisterAsSourceForSceneGraph(cam_scene_graph)
        parser = Parser(cam_plant)
        parser.AddModels(camera_files[i]) 
        builder.AddSystem(cam_scene_graph)

        renderer = None
        depth_camera = None

        if not renderer:
            renderer = "my_renderer"

        if not scene_graph.HasRenderer(renderer):
            scene_graph.AddRenderer(renderer, MakeRenderEngineVtk(RenderEngineVtkParams()))

        if not depth_camera:
            depth_camera = DepthRenderCamera(RenderCameraCore(renderer, CameraInfo(width=640, height=480, fov_y=np.pi / 4.0), ClippingRange(near=0.1, far=10.0), RigidTransform()),DepthRange(0.1, 10.0))

        inspector = scene_graph.model_inspector()
        ids = inspector.GetAllFrameIds()

        rgbd = builder.AddSystem(RgbdSensor(parent_id = ids[0], X_PB=RigidTransform(), depth_camera=depth_camera, show_window=False))
        model_name = "cam"+str(i)
        rgbd.set_name(model_name)
        builder.Connect(scene_graph.get_query_output_port(),rgbd.query_object_input_port())

        # Export the camera outputs
        builder.ExportOutput(rgbd.color_image_output_port(), f"{model_name}_rgb_image")
        builder.ExportOutput(rgbd.depth_image_32F_output_port(), f"{model_name}_depth_image")
        builder.ExportOutput(rgbd.label_image_output_port(), f"{model_name}_label_image")

    # Setup slider input for pendulum input torque
    meshcat.AddSlider("u", min=-5, max=5, step=0.1, value=0.0)
    torque_system = builder.AddSystem(MeshcatSliders(meshcat, ["u"]))
    builder.Connect(torque_system.get_output_port(), pendulum.get_input_port())

    # Add visualizer to visualize the geometries.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat)

    diagram = builder.Build()
    return diagram, visualizer

In [ ]:
def run_simulation(x_traj_source, sim_time_step):
    diagram, visualizer = create_scene(x_traj_source, sim_time_step)
    simulator = initialize_simulation(diagram)
    visualizer.StartRecording()
    simulator.AdvanceTo(2)
    visualizer.PublishRecording()

run_simulation(x_traj_source, 0.0001)

# Demos

The following links contain demos of the final drone simulation

Demo link 1: https://youtu.be/Yer-eJaSbwc

Demo link 2: https://youtu.be/RP1e5ff_Fc8

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d095d7c3-1038-4cc5-bfcb-77f17ef5665a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>